# Homework 6: Exact Diagonalization and ODE
You can use `scipy.linalg` functions to solve these problems unless 

## Harmonic Oscillator

<img src="HW6_Fig1.jpg" height="100"  alt="Coupled Springs">

The magnetized iron block of mass $m$ is attached to a spring of stiffness $k$ and free length $L$. 
The block is at rest at $x=L$ when the electromagnet is turned on, exerting the repulsive force $F=c / x^2$ on the block. The differential equation of the resulting motion is

$$
m \ddot{x}=\frac{c}{x^2}-k(x-L)
$$

### TODO
Determine the period of the ensuing motion by numerical integration with the adaptive Runge-Kutta method. Use $c=5 \mathrm{~N} \cdot \mathrm{~m}^2, k=120 \mathrm{~N} / \mathrm{m}, L=0.2 \mathrm{~m}$, and $m=1.0 \mathrm{~kg}$.



## Exact Diagonalization of a 1D Heisenberg Chain

Translational symmetry allows us to construct the momentum basis for a given reference state $|a\rangle$, 
$$
|a(k)\rangle= \frac{1}{\sqrt{N_a}} \sum_{r=0}^{N-1} e^{-ikr} T^r |a\rangle,
$$
where the translation operator $T$ shift the spins one step cyclically to the right, 
$$
T|S^z_0, S^z_1, \cdots, S^z_{N-1}\rangle=|S^z_{N-1}, S^z_0,\cdots, S^z_{N-2}\rangle,
$$
and $N_a$ is a normalization factor.
The momenta are given by 
$$
k=m \frac{2\pi}{L}, \quad m=-L/2+1, \ldots, L/2
$$

### TODO
1. Modify the code below to add the translation symmetry. Find the ground states for $k=0,\pi$.
2. Compare your results with the results from [QuSpin](https://quspin.github.io/QuSpin/) library.

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh

def construct_hamiltonian(L, J):
    """
    Constructs the Heisenberg Hamiltonian for a spin-1/2 chain of length L with periodic boundary conditions.
    The Hamiltonian is built in the Sz=0 sector.
    """
    # Generate all possible spin configurations as integers
    states = []
    for state in range(2**L):
        spin_sum = sum(1 if (state >> i) & 1 else -1 for i in range(L))
        if spin_sum == 0:  # Select only states with Sz = 0
            states.append(state)

    # Mapping of states to basis indices
    state_to_index = {state: idx for idx, state in enumerate(states)}
    dim = len(states)

    # Sparse matrix storage for the Hamiltonian
    rows, cols, data = [], [], []

    # Construct the Hamiltonian
    for state in states:
        i = state_to_index[state]

        # Iterate over all pairs of neighboring spins
        for site in range(L):
            # Sites to interact
            spin1 = (state >> site) & 1
            spin2 = (state >> ((site + 1) % L)) & 1

            # ZZ term
            zz = (1 if spin1 == spin2 else -1)
            rows.append(i)
            cols.append(i)
            data.append(J * zz * 0.25)

            # XX and YY terms (off-diagonal)
            if spin1 != spin2:
                # Flip spins
                flipped_state = state ^ (1 << site) ^ (1 << ((site + 1) % L))
                j = state_to_index[flipped_state]
                rows.append(i)
                cols.append(j)
                data.append(J * 0.5 * 0.25)

    # Build sparse Hamiltonian matrix
    H = csr_matrix((data, (rows, cols)), shape=(dim, dim), dtype=np.float64)
    return H, states

def heisenberg_ground_state(L, J=1.0):
    """
    Computes the ground state energy and wavefunction of a spin-1/2 Heisenberg chain of length L
    with periodic boundary conditions, in the Sz=0 symmetry sector.
    """
    H, states = construct_hamiltonian(L, J)

    # Find the lowest eigenvalue and eigenvector using sparse diagonalization
    eigenvalues, eigenvectors = eigsh(H, k=1, which='SA')

    ground_state_energy = eigenvalues[0]
    ground_state_wavefunction = eigenvectors[:, 0]

    return ground_state_energy, ground_state_wavefunction, states

# Parameters
L = 6  # Length of the chain
J = 1.0  # Exchange interaction strength

# Compute ground state
energy, wavefunction, states = heisenberg_ground_state(L, J)

print("Ground State Energy:", energy)
print("Ground State Wavefunction:", wavefunction)


In [2]:
!pip install quspin

from quspin.operators import hamiltonian  # Hamiltonian construction
from quspin.basis import spin_basis_1d    # Spin basis for 1d chains
import numpy as np

# Parameters
L = 8  # Length of the spin chain
J = 1.0  # Exchange interaction strength (Jx = Jy = Jz = J)

# Create spin basis with Sz symmetry
basis = spin_basis_1d(L, pauli=False, kblock=0, pblock=1)

# Define the Heisenberg Hamiltonian with periodic boundary conditions
J_list = [[J, i, (i + 1) % L] for i in range(L)]  # Coupling terms (i, i+1)

static = [["xx", J_list], ["yy", J_list], ["zz", J_list]]
H = hamiltonian(static, [], basis=basis, dtype=np.float64)

# Diagonalize the Hamiltonian to find the ground state
eigenvalues, eigenvectors = H.eigh()

# Extract the ground state energy and wavefunction
ground_state_energy = eigenvalues[0]
ground_state_wavefunction = eigenvectors[:, 0]

# Print results
print("Ground State Energy:", ground_state_energy)
print("Ground State Wavefunction:", ground_state_wavefunction)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 865.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.2 MB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of contourpy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.9/199.9 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 1.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 2.6 